In [4]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split


# Importing ML models 
from catboost import CatBoostClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
df_FCh_descr = pd.read_csv()
df_MoFP_descr = pd.read_csv()
df_FCh_MoFP_descr = pd.read_csv()